In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Transformer
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql.functions import col, sin, cos, radians
from pyspark.sql import DataFrame
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
team = 19
warehouse = "project/warehouse"

In [2]:
spark = SparkSession.builder\
        .appName("{} - spark ML".format(team))\
        .master("yarn")\
        .config("hive.metastore.uris", "thrift://hadoop-02.uni.innopolis.ru:9883")\
        .config("spark.sql.warehouse.dir", warehouse)\
        .config("spark.sql.avro.compression.codec", "snappy")\
        .enableHiveSupport()\
        .getOrCreate()

In [3]:
for table in spark.catalog.listTables("team19_projectdb"):
    print(table.name)

air_quality
astronomical_data_part
locations
q1_results
q2_results
q4_results
q5_results
weather_conditions


In [4]:
air_quality = spark.read.format("avro").table('team19_projectdb.air_quality')
astronomical_data = spark.read.format("avro").table('team19_projectdb.astronomical_data_part')
locations = spark.read.format("avro").table('team19_projectdb.locations')
weather_conditions = spark.read.format("avro").table('team19_projectdb.weather_conditions')

In [ ]:
table_for_ML = air_quality.join(locations, on=["id"], how="inner").join(weather_conditions, on=["id"], how="inner")

In [20]:
table_for_ML.printSchema()

root
 |-- id: integer (nullable = true)
 |-- air_quality_carbon_monoxide: double (nullable = true)
 |-- air_quality_ozone: double (nullable = true)
 |-- air_quality_nitrogen_dioxide: double (nullable = true)
 |-- air_quality_sulphur_dioxide: double (nullable = true)
 |-- air_quality_pm2_5: double (nullable = true)
 |-- air_quality_pm10: double (nullable = true)
 |-- air_quality_us_epa_index: integer (nullable = true)
 |-- air_quality_gb_defra_index: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: string (nullable = true)
 |-- last_updated_epoch: integer (nullable = true)
 |-- last_updated: long (nullable = true)
 |-- temperature_celsius: double (nullable = true)
 |-- temperature_fahrenheit: double (nullable = true)
 |-- condition_text: string (nullable = true)
 |-- wind_mph: double (nullable = true)
 |-- wind_kph: double (nullable

In [24]:
class LatLongToECEF(Transformer, DefaultParamsReadable, DefaultParamsWritable):
    def __init__(self):
        super(LatLongToECEF, self).__init__()

    def _transform(self, df: DataFrame) -> DataFrame:
        # WGS84 constants
        a = 6378137.0  # semi-major axis
        e2 = 6.6943799901377997e-3

        # Convert degrees to radians
        lat_rad = radians(col("latitude"))
        lon_rad = radians(col("longitude"))

        # Calculate ECEF coordinates
        N = a / (1 - e2 * sin(lat_rad) ** 2) ** 0.5

        X = N * cos(lat_rad) * cos(lon_rad)
        Y = N * cos(lat_rad) * sin(lon_rad)
        Z = (1-e2) * N * sin(lat_rad)

        return df.withColumn("x", X).withColumn("y", Y).withColumn("z", Z)


ecof_transformer = LatLongToECEF()

In [25]:
ecof_transformer.transform(table_for_ML).head(1)

[Row(id=1, air_quality_carbon_monoxide=647.5, air_quality_ozone=130.2, air_quality_nitrogen_dioxide=1.2, air_quality_sulphur_dioxide=0.4, air_quality_pm2_5=7.9, air_quality_pm10=11.1, air_quality_us_epa_index=1, air_quality_gb_defra_index=1, country='Afghanistan', location_name='Kabul', latitude=34.52, longitude=69.18, timezone='Asia/Kabul', last_updated_epoch=1693301400, last_updated=1693306800000, temperature_celsius=28.8, temperature_fahrenheit=83.8, condition_text='Sunny', wind_mph=7.2, wind_kph=11.5, wind_degree=74, wind_direction='ENE', pressure_mb=1004.0, pressure_in=29.64, precip_mm=0.0, precip_in=0.0, humidity=19, cloud=0, feels_like_celsius=26.7, feels_like_fahrenheit=80.1, visibility_km=10.0, visibility_miles=6.0, uv_index=7.0, gust_mph=8.3, gust_kph=13.3, x=1869858.4270867545, y=4917265.761836958, z=3594120.1546326494)]

In [37]:
# target is string, so cast to int
indexer = StringIndexer(inputCol="condition_text", outputCol="target")

In [38]:
indexer.fit(table_for_ML).transform(table_for_ML).head(1)

[Row(id=1, air_quality_carbon_monoxide=647.5, air_quality_ozone=130.2, air_quality_nitrogen_dioxide=1.2, air_quality_sulphur_dioxide=0.4, air_quality_pm2_5=7.9, air_quality_pm10=11.1, air_quality_us_epa_index=1, air_quality_gb_defra_index=1, country='Afghanistan', location_name='Kabul', latitude=34.52, longitude=69.18, timezone='Asia/Kabul', last_updated_epoch=1693301400, last_updated=1693306800000, temperature_celsius=28.8, temperature_fahrenheit=83.8, condition_text='Sunny', wind_mph=7.2, wind_kph=11.5, wind_degree=74, wind_direction='ENE', pressure_mb=1004.0, pressure_in=29.64, precip_mm=0.0, precip_in=0.0, humidity=19, cloud=0, feels_like_celsius=26.7, feels_like_fahrenheit=80.1, visibility_km=10.0, visibility_miles=6.0, uv_index=7.0, gust_mph=8.3, gust_kph=13.3, target=2.0)]

In [29]:
features = ["air_quality_carbon_monoxide", "air_quality_ozone", "air_quality_nitrogen_dioxide", "air_quality_pm2_5", "air_quality_pm10" , "air_quality_us_epa_index", "air_quality_gb_defra_index",\
            "temperature_celsius", "wind_kph", "wind_degree", "pressure_mb", "precip_mm", "humidity", "cloud", "feels_like_celsius", "visibility_km", "gust_mph", "x", "y", "z"]

label = "target"

In [36]:
pipeline = Pipeline(stages=[indexer, ecof_transformer])

In [39]:
model = pipeline.fit(table_for_ML)
data = model.transform(table_for_ML)

In [41]:
(train_data, test_data) = data.randomSplit([0.6, 0.4], seed = 10)